In [322]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mlp
#verify mlp

In [323]:
df = pd.read_excel('sales.xlsx')

In [324]:
df.head(3)

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza


In [325]:
df.shape

(48620, 12)

In [326]:
df.columns

Index(['order_details_id', 'order_id', 'pizza_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')

In [327]:
df.dtypes

order_details_id              int64
order_id                      int64
pizza_id                     object
quantity                      int64
order_date           datetime64[ns]
order_time                   object
unit_price                  float64
total_price                 float64
pizza_size                   object
pizza_category               object
pizza_ingredients            object
pizza_name                   object
dtype: object

# Questions

## 2. Informations de base :

a. Combien de pizzas uniques (pizza_id distincts) y a-t-il dans le jeu de
données ?

In [328]:

df.pizza_id.value_counts().count()

91

b.  Combien de commandes uniques (order_id distincts) figurent dans le jeu
de données ?

In [329]:
df.order_id.value_counts().count()

21350

## 3. Analyse temporelle :

a. Quelle est la plage de dates dans le jeu de données ?

In [330]:
print(f"plage de dates : {df['order_date'].min()} to {df['order_date'].max()}")

plage de dates : 2015-01-01 00:00:00 to 2015-12-31 00:00:00


b. Combien de commandes ont été passées chaque jour ?

In [331]:
df.groupby('order_date')['order_id'].count()

order_date
2015-01-01    161
2015-01-02    160
2015-01-03    154
2015-01-04    106
2015-01-05    121
             ... 
2015-12-27     87
2015-12-28    102
2015-12-29     77
2015-12-30     73
2015-12-31    171
Name: order_id, Length: 358, dtype: int64

c. Quelle est la durée moyenne pour traiter une commande ?

C'est une question piège nous n'avons aucun moyen de calculer correctement cette valeur, il faudrait que le Dataframe contienne une colonne order_processed ou autre pour pouvoir faire ce calcul.

Une très mauvaise solution serait de calculer la différence de temps passée entre chaque commandes triées par order_id (1 et 2 par ex) pour essayer de trouver cette valeur. Cela ne serait pas très pertinent car les résultats ne seront pas nécessairement représentatif du temps que Jo va prendre pour traiter une commande, et si il prend une pause, qu'il n'a pas de clients pendant un moment ou ferme la pizzaria après une commande, le calcul va considérer cela comme du temps de préparation de la commande et fausser les résultats

## 4. Analyse de la quantité :

a. Quelle est la quantité moyenne de pizzas commandées ?

In [332]:
df.groupby('order_id')['quantity'].sum().mean()

2.321967213114754

Values Check

In [333]:
# You can check the results here
# df.quantity.value_counts()
# df[df.quantity == 4]
# df.groupby('order_id')['quantity'].sum()[5264]
# df[df.order_id == 5264]

b. Quelle est la taille de pizza la plus fréquemment commandée ?

In [334]:
df.pizza_size.value_counts()

pizza_size
L      18526
M      15385
S      14137
XL       544
XXL       28
Name: count, dtype: int64

In [335]:
df.pizza_size.value_counts().index[0]

'L'

c. Quelle est la catégorie de pizza la plus fréquemment commandée ?

In [336]:
df.pizza_category.value_counts()

pizza_category
Classic    14579
Supreme    11777
Veggie     11449
Chicken    10815
Name: count, dtype: int64

In [337]:
df.pizza_category.value_counts().index[0]

'Classic'

## 5. Analyse des prix :

a. Quel est le prix unitaire moyen des pizzas ?

In [338]:
df.unit_price.mean()

16.494132044426163

b. Quel est le prix total moyen d'une commande ?

In [339]:
df.groupby('order_id')['unit_price'].sum().mean()

37.56181264637002

c. Quelle pizza a le prix unitaire le plus élevé ?

In [340]:
# Je vérifies d'abord qu'il y a bien une seule pizza qui à le prix le plus élevé
df[df.unit_price == df.unit_price.max()]['pizza_name'].value_counts().count()

# ceci donne le bon nom de pizza mais on peut faire un peu mieux (case en dessous)
# df[df.unit_price == df.unit_price.max()]['pizza_name'].value_counts()[0]

1

In [341]:
# plus fluide :
df[df.unit_price == df.unit_price.max()].iloc[0]['pizza_name']

'The Greek Pizza'

## 6. Répartition par taille et catégorie :

a. Combien de pizzas de chaque taille ont été vendues ?

In [342]:
df.groupby('pizza_size')['quantity'].sum()

pizza_size
L      18956
M      15635
S      14403
XL       552
XXL       28
Name: quantity, dtype: int64

b. Combien de pizzas y a-t-il dans chaque catégorie ?

In [343]:
df.groupby('pizza_category')['pizza_name'].nunique()

pizza_category
Chicken    6
Classic    8
Supreme    9
Veggie     9
Name: pizza_name, dtype: int64

In [344]:
# Une bien pire façon de l'afficher
df.groupby('pizza_category')['pizza_name'].value_counts().index

MultiIndex([('Chicken',                 'The Barbecue Chicken Pizza'),
            ('Chicken',                     'The Thai Chicken Pizza'),
            ('Chicken',               'The California Chicken Pizza'),
            ('Chicken',                'The Southwest Chicken Pizza'),
            ('Chicken',                  'The Chicken Alfredo Pizza'),
            ('Chicken',                    'The Chicken Pesto Pizza'),
            ('Classic',                   'The Classic Deluxe Pizza'),
            ('Classic',                         'The Hawaiian Pizza'),
            ('Classic',                        'The Pepperoni Pizza'),
            ('Classic',                         'The Big Meat Pizza'),
            ('Classic',                       'The Napolitana Pizza'),
            ('Classic',                'The Italian Capocollo Pizza'),
            ('Classic',                            'The Greek Pizza'),
            ('Classic', 'The Pepperoni, Mushroom, and Peppers Pizza'),
      

## 7. Analyse des ingrédients :

a. Quels sont les ingrédients les plus couramment utilisés dans toutes les
pizzas ?


In [345]:
# avoiding repeating operation error by checking if last element of the
# column has already been converted to a list instead of string
if not isinstance(df.pizza_ingredients[df.pizza_ingredients.shape[0]-1], list):
    # cleaning list of ingredients
    df.pizza_ingredients = df.pizza_ingredients.apply(lambda x : x.split(', '))
ingredients_count = dict(df.pizza_ingredients.explode().value_counts())
ingredients_count

AttributeError: 'list' object has no attribute 'trim'

In [ ]:
ingredients_count

pizza_ingredients
 Garlic                27422
 Tomatoes              23694
 Red Onions            19547
 Red Peppers           16284
Chicken                 8443
                       ...  
Brie Carre Cheese        480
 Prosciutto              480
 Caramelized Onions      480
 Pears                   480
 Thyme                   480
Name: count, Length: 73, dtype: int64

b. Combien de pizzas contiennent un ingrédient spécifique (par exemple,
pepperoni, champignons) ?

In [ ]:
def HowManyPizzsUseThisIngredient(df, ingredient):
    p



HowManyPizzsUseThisIngredient(df,)

## 8. Détails de la commande :

a. Quelle est la répartition du nombre de pizzas par commande ?

b. Quelle est la répartition des prix totaux pour les commandes ?

## 9. Analyse du chiffre d’affaires en fonction du temps :

a. Quel est le chiffre d'affaires quotidien ?


b. Y a-t-il une corrélation entre l'heure de la journée et le total de la
commande ?

## 10.Pizzas populaires :

a. Quelles sont les 5 pizzas les plus commandées en quantité ?

b. Quelles sont les 5 pizzas les plus commandées en termes de chiffre
d'affaires ?

## 11.Chiffre d'affaires par taille et catégorie :

a. Quelle est la répartition du chiffre d'affaires pour chaque taille de pizza ?

b. Quelle est la répartition du chiffre d'affaires pour chaque catégorie de
pizza ?

## 12.Corrélation entre prix et quantité :

a. Y a-t-il une corrélation entre le prix unitaire et la quantité commandée ?

b. Quel est le prix unitaire moyen pour différentes tailles de pizza ?

## 13.Tendances saisonnières :

a. Y a-t-il des tendances saisonnières dans les commandes de pizza ?

b. Y a-t-il une corrélation entre le jour de la semaine et la quantité de
commandes ?

In [ ]:
# 5. Analyse des prix :
# a. Quel est le prix unitaire moyen des pizzas ?
# b. Quel est le prix total moyen d'une commande ?
# c. Quelle pizza a le prix unitaire le plus élevé ?
# 6. Répartition par taille et catégorie :
# a. Combien de pizzas de chaque taille ont été vendues ?
# b. Combien de pizzas y a-t-il dans chaque catégorie ?
# 7. Analyse des ingrédients :
# a. Quels sont les ingrédients les plus couramment utilisés dans toutes les
# pizzas ?
# b. Combien de pizzas contiennent un ingrédient spécifique (par exemple,
# pepperoni, champignons) ?
# 8. Détails de la commande :
# a. Quelle est la répartition du nombre de pizzas par commande ?
# b. Quelle est la répartition des prix totaux pour les commandes ?
# 9. Analyse du chiffre d’affaires en fonction du temps :
# a. Quel est le chiffre d'affaires quotidien ?
# b. Y a-t-il une corrélation entre l'heure de la journée et le total de la
# commande ?
# 10.Pizzas populaires :
# a. Quelles sont les 5 pizzas les plus commandées en quantité ?

# b. Quelles sont les 5 pizzas les plus commandées en termes de chiffre
# d'affaires ?

# 11.Chiffre d'affaires par taille et catégorie :
# a. Quelle est la répartition du chiffre d'affaires pour chaque taille de pizza ?
# b. Quelle est la répartition du chiffre d'affaires pour chaque catégorie de
# pizza ?

# 12.Corrélation entre prix et quantité :
# a. Y a-t-il une corrélation entre le prix unitaire et la quantité commandée ?
# b. Quel est le prix unitaire moyen pour différentes tailles de pizza ?
# 13.Tendances saisonnières :
# a. Y a-t-il des tendances saisonnières dans les commandes de pizza ?
# b. Y a-t-il une corrélation entre le jour de la semaine et la quantité de
# commandes ?

In [ ]:
# 1. Installez et importez Pandas.
# 2. Informations de base :


# a. Combien de pizzas uniques (pizza_id distincts) y a-t-il dans le jeu de
# données ?
# b. Combien de commandes uniques (order_id distincts) figurent dans le jeu
# de données ?

# 3. Analyse temporelle :
# a. Quelle est la plage de dates dans le jeu de données ?
# b. Combien de commandes ont été passées chaque jour ?
# c. Quelle est la durée moyenne pour traiter une commande ?


# 4. Analyse de la quantité :
# a. Quelle est la quantité moyenne de pizzas commandées ?
# b. Quelle est la taille de pizza la plus fréquemment commandée ?
# c. Quelle est la catégorie de pizza la plus fréquemment commandée ?

# The End

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# ➔ Quels sont les jours et les heures où nous sommes le plus occupés ?
# df.order_date.apply(lambda x : x.day)

# ➔ Combien de pizzas fabriquons-nous pendant les périodes de pointe ?
# df.order_time.apply(lambda x : np.mean(x))
# ➔ Quelles sont les pizzas les plus vendues et les moins vendues ?

# ➔ Quelle est la valeur moyenne de nos commandes ?
